In [1]:
import importlib
import corona_plots
importlib.reload(corona_plots)
import daily
import importlib
importlib.reload(daily)

import pandas as pd
import os
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go
import plotly
hex_colors = [
    '#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2',
    '#7f7f7f','#bcbd22','#17becf','#7f0000','#000000','#04ab08','#0000fb',
]


In [2]:
start=pd.Timestamp(2020, 1, 22)
end=pd.Timestamp(2020, 4, 8)

In [3]:
df, last = daily.DailyData.ranged_country(start=start, end=end, indicator='Confirmed')
df2, _ =   daily.DailyData.ranged_country(start=start, end=end, indicator='Deaths')
print('ddd')

fig = make_subplots(rows=3, cols=1, vertical_spacing=0.02, row_heights=[0.45, 0.45, 0.1], shared_xaxes=True)

dfnews = daily.DailyData.news_dataframe()
fig.add_trace(
    go.Scatter(
        x=dfnews.date, y=[0] * len(dfnews.date), mode='markers', name="News",
        legendgroup="News", marker=dict(color='#d62728', size=10), marker_symbol=['diamond-dot']*len(dfnews.date),
        text=[f'{body}' for body in dfnews.body]
    ), row=3, col=1
)
for idx, (total, col) in enumerate(last):
    if col == 'Recovered':
        continue
    if total < 100:
        continue
    color = hex_colors[idx % len(hex_colors)]    
    count = list(df[col])
    fig.add_trace(
        go.Scatter(
            x=df.index, y=count, mode='lines+markers', name=f'{col} last total: {int(total)}',
            legendgroup=col, line=dict(color=color), marker=dict(color=color),
        ), row=1, col=1
    )
    count_diff = np.array(count) - np.array([0]+count[:-1])
    fig.add_trace(
        go.Scatter(
            x=df.index, y=count_diff, mode='lines+markers', name=f'{col} last daily: {int(count_diff[-1])}',
            legendgroup=col, line=dict(color=color), marker=dict(color=color),
        ), row=2, col=1
    )
fig.update_yaxes(title_text="Total Confirmed", row=1, col=1)
fig.update_yaxes(title_text="Daily Confirmed", row=2, col=1)
fig.update_yaxes(title_text="News", row=3, col=1)

plotly.offline.plot(fig, filename = 'global.html', auto_open=True)

ddd


'corona_global.html'

In [9]:
df, last = daily.DailyData.ranged_region(start=start, end=end, country='US', indicator='Confirmed')
df2, _ =   daily.DailyData.ranged_region(start=start, end=end, country='US', indicator='Deaths')
fig = go.Figure()
for idx, (total, col) in enumerate(last):
    if col == 'Recovered':
        continue
    color = hex_colors[idx % len(hex_colors)]    
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df2[col], mode='lines+markers', name=f'{col}: {int(total)}',
            legendgroup=col, line=dict(color=color), marker=dict(color=color),
        )
    )
#fig.update_layout(xaxis_type="log", yaxis_type="log")
fig
plotly.offline.plot(fig, filename = 'us_death.html', auto_open=True)

'us_death.html'

In [5]:
df, last = daily.DailyData.ranged_region(start=start, end=end, country='US', indicator='Confirmed')
df2, _ =   daily.DailyData.ranged_region(start=start, end=end, country='US', indicator='Deaths')
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.05)
for idx, (total, col) in enumerate(last):
    if col == 'Recovered':
        continue
    color = hex_colors[idx % len(hex_colors)]
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df[col], mode='lines+markers', name=f'{col}: {int(total)}',
            legendgroup=col, line=dict(color=color), marker=dict(color=color),
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=df2.index, y=df2[col], mode='lines+markers', name=f'{col}: {int(total)}',
            legendgroup=col, line=dict(color=color), marker=dict(color=color),
        ),
        row=2, col=1
    )


fig.update_layout(title="Corona virus tracking: USA confirmed v.s deaths accross states")
fig.update_yaxes(title_text="Accumulated positive count", row=1, col=1)
fig.update_yaxes(title_text="Accumulated death count", row=2, col=1)
plotly.offline.plot(fig, filename = 'corona_us.html', auto_open=True)

'corona_us.html'

In [6]:
df, last = daily.DailyData.ranged_region(start=start, end=end, country='China', indicator='Confirmed')
df2, _ =   daily.DailyData.ranged_region(start=start, end=end, country='China', indicator='Deaths')
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.05)
for idx, (total, col) in enumerate(last):
    if col == 'Recovered':
        continue
    color = hex_colors[idx % len(hex_colors)]
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df[col], mode='lines+markers', name=f'{col}: {int(total)}',
            legendgroup=col, line=dict(color=color), marker=dict(color=color),
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=df2.index, y=df2[col], mode='lines+markers', name=f'{col}: {int(total)}',
            legendgroup=col, line=dict(color=color), marker=dict(color=color),
        ),
        row=2, col=1
    )


fig.update_layout(title="Corona virus tracking: China confirmed v.s deaths accross states")
fig.update_yaxes(title_text="Accumulated positive count", row=1, col=1)
fig.update_yaxes(title_text="Accumulated death count", row=2, col=1)
plotly.offline.plot(fig, filename = 'corona_china.html', auto_open=True)

'corona_china.html'